<div style='background:#4a565e; padding:5px; margin:0'> 
    <div style=''>
        <img src="https://www.inegi.org.mx/img/home/logoinegibco.png" alt="logo inegi" />
    </div>
</div>

## Unir información con la malla estadistica

In [ ]:
import requests
import os
import zipfile
import geopandas as gpd 
import pandas as pd

* ruta donde se encuentran los archivos

In [ ]:
path="../01-Data/01_data_geostatistical_grid"

* Malla geoestadística

In [ ]:
gdf_malla = gpd.read_file(path+"/mallaGeo.zip!INEGI_CPV2020_n9_.shp").to_crs(4326)

* Copernicus

In [ ]:
def CopernicusCPV(gdf_copernicus,field,new_field):
  interseccion = gdf_copernicus.sjoin(gdf_malla, how="left")
  resultado = interseccion.groupby("CODIGO")[field].apply(lambda x: x.value_counts().idxmax()).reset_index()
  resultado = resultado.rename(columns={field: new_field})
  return resultado

In [ ]:
gdf1b = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_builtUpA_v2.shp")
gdf2b = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_builtUpA_v2.shp")
gdf_built = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1b, gdf2b], ignore_index=True)),"damage_gra","built")
gdf1f = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_observedEventA_v2.shp")
gdf2f = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_observedEventA_v2.shp")
gdf_flood = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1f, gdf2f], ignore_index=True)),"notation","flooded")
gdf1L = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_transportationL_v2.shp")
gdf2L = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_transportationL_v2.shp")
gdf_transL = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1L, gdf2L], ignore_index=True)),"damage_gra","transpor")
gdf1p = gpd.read_file(path+"/EMSR703_AOI01_GRA_MONIT01_v2.zip!EMSR703_AOI01_GRA_MONIT01_transportationP_v2.shp")
gdf2p = gpd.read_file(path+"/EMSR703_AOI02_GRA_MONIT01_v2.zip!EMSR703_AOI02_GRA_MONIT01_transportationP_v2.shp")
gdf_transp = CopernicusCPV(gpd.GeoDataFrame(pd.concat([gdf1p, gdf2p], ignore_index=True)),"damage_gra","transpor")

In [ ]:
gdf_trans = gpd.GeoDataFrame(pd.concat([gdf_transL, gdf_transp], ignore_index=True))
gdf_Copern = gdf_built.merge(gdf_flood, on='CODIGO', how='outer').merge(gdf_trans, on='CODIGO', how='outer')

* DENUE

In [ ]:
%time gdf_DENUE = gpd.read_file(path+"/denue_12_shp.zip!conjunto_de_datos/denue_inegi_12_.shp").to_crs(4326)

CPU times: user 1min 43s, sys: 1.5 s, total: 1min 44s
Wall time: 1min 44s


In [ ]:
interseccionDENUE = gpd.sjoin(gdf_DENUE, gdf_malla, how="left")
interseccionDENUE['sec']  = interseccionDENUE['codigo_act'].astype(str).str[:2]
inter = interseccionDENUE[['sec', 'CODIGO']]
gdf_DENUEcount = inter.groupby(['CODIGO', 'sec']).size().reset_index(name='count')
gdf_DENUE = gdf_DENUEcount.pivot(index='CODIGO', columns='sec', values='count')
gdf_DENUE = gdf_DENUE.fillna(0)

* Charter activations

In [ ]:
gdf_Flooding = gpd.read_file(path+"/vap-974-1-product.geojson")
intersecFlooding = gpd.sjoin(gdf_Flooding, gdf_malla, how="left")
df_CA_Flooding = intersecFlooding.groupby(['CODIGO'])['DN'].count().reset_index(name='CA_flood')

In [ ]:
gdf_Slides = gpd.read_file(path+"/GeoJSON-file-for-Landslides-Mexico")
intersecSlides = gpd.sjoin(gdf_Slides, gdf_malla, how="left")
df_CA_Slides = intersecSlides.groupby(['CODIGO'])['DN'].count().reset_index(name='CA_Slides')

## Unir datos con la malla geoestadística
Guardamos el resultado en un nuevo archivo Shapefile

In [ ]:
ret_res1= gdf_malla.merge(gdf_Copern, on='CODIGO')
ret_res2= ret_res1.merge(gdf_DENUE, on='CODIGO')
ret_res3= ret_res2.merge(df_CA_Flooding, on='CODIGO')
ret_res4= ret_res3.merge(df_CA_Slides, on='CODIGO')
ret_res4.to_file("01_grid/grid_integration.shp")

<div style='background:#4a565e; padding:5px; margin:0'> 
    <img src="https://cienciadedatos.inegi.org.mx/laboratorio/img/icono_lcid.jpg" alt="LCiD" width="30"/>
    <div style="color:#ccd6de; float:right" >
        Laboratorio de ciencia de datos y métodos modernos de Producción de información |
        <a href="mailto:lcid@inegi.org.mx">lcid@inegi.org.mx</a>
    </div>
</div>